In [2]:
import os
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
import re

import evaluate
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

# konlpy
import konlpy
from konlpy.tag import Okt

# Seed Set
SEED = 456
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

# 디바이스 설정 (GPU가 사용 가능하면 GPU를 사용하고, 그렇지 않으면 CPU 사용)
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# ASCII-code 기반으로 노이즈 체크

In [3]:
data_path = os.path.join('..', '..', 'data')
data = pd.read_csv(os.path.join(data_path,"train.csv"))
print(data_path)

../../data


In [4]:
data.head()

,ID,text,target
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5
4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6


In [5]:
text_datas = data["text"]
text_datas

0        정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보
1             K찰.국DLwo 로L3한N% 회장 2 T0&}송=
2                  m 김정) 자주통일 새,?r열1나가야1보
3           갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩
4            pI美대선I앞두고 R2fr단 발] $비해 감시 강화
                      ...                
2795    트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도
2796          삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠
2797     텔레그램+한D 등h亞서 2시간H다운…C버T정gf39종!2보
2798      인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정
2799                 지능정보사회 대비 국가 종합대책 마련
Name: text, Length: 2800, dtype: object

In [6]:
def calculate_ascii_noise(text):
    total_chars = len(text)
    ascii_chars = 0 
    for ch in text:
        if ch == " ":
            continue
        if ord(ch) < 128:
            ascii_chars+=1
            
    non_ascii_chars = total_chars - ascii_chars  # 비-ASCII 문자 개수
    
    ascii_ratio = (ascii_chars / total_chars) * 100  # ASCII 문자 비율 (%)
    
    return ascii_chars, non_ascii_chars, ascii_ratio

In [7]:
asc_counts = []; non_asc_counts = []; percentages = []
# ASCII 문자 개수, 비-ASCII 문자 개수, ASCII 비율 계산
for text_with_noise in text_datas:
    ascii_count, non_ascii_count, ascii_percentage = calculate_ascii_noise(text_with_noise)
    ascii_percentage = round(ascii_percentage,2)
    asc_counts.append(ascii_count)
    non_asc_counts.append(non_ascii_count)
    percentages.append(ascii_percentage)
    
print(len(asc_counts), len(non_asc_counts), len(percentages))

2800 2800 2800


In [8]:
data["ascii_counts"] = asc_counts
data["non_ascii_counts"] = non_asc_counts
data["ratio"] = percentages

In [19]:
Noise_data = data[data["ratio"]<30]
Noise_data[Noise_data["ratio"]>25]
# Noise_data.to_csv(os.path.join(data_path,"train_Noise_data_ascii.csv"),encoding="utf-8-sig")

,ID,text,target,ascii_counts,non_ascii_counts,ratio
33,ynat-v1_train_00033,학~ 석면 S거 규정k지키5,3,4,11,26.67
66,ynat-v1_train_00066,UEFA 챔스리그 4강 마드리드 더비 성사…레알 vs 아...,0,10,24,29.41
76,ynat-v1_train_00076,北조국통!민주X#전선 결성 <0돌 중;보고회 열x,2,7,20,25.93
126,ynat-v1_train_00126,날씨-미z먼지 보통 수y…오후부터 xJ지고:곳곳 F12일,0,9,22,29.03
138,ynat-v1_train_00138,삼성 빅스. 개발자데KS1B월 서울B 첫 +최,4,7,18,28.00
...,...,...,...,...,...,...
2739,ynat-v1_train_02739,:한금투 자동차 업/ 1분기부k5실적 점진?|I선,5,8,19,29.63
2741,ynat-v1_train_02741,~ 전 앞둔eW재 감98T이 부K되지만 4로 승부수,1,8,21,27.59
2745,ynat-v1_train_02745,軍v<들 전용3량 타고 골l장y…`덕적 해R 심각,3,7,20,25.93
2780,ynat-v1_train_02780,"北0대! 김정*2향한 ""성경쟁 무대된l릴레이r토론",2,7,20,25.93


In [68]:
Label_noise_data = data[data["ratio"]<30]
Label_noise_data.to_csv(os.path.join(data_path,"train_Label_noise_data_ascii.csv"),encoding="utf-8-sig")

In [67]:
data_30p = data[data["ratio"]<30.0].reset_index(drop=True).to_csv(os.path.join(data_path,"train_30p_ASCII_removed.csv"),encoding="utf-8-sig")

In [194]:
text_datas[text_datas["ratio"]<35.0].iloc[29]

ID                              ynat-v1_train_00090
text                서울문고 N풍그U에 인수… f라인서점 교m영풍y양강d도로
ascii_counts                                     10
non_ascii_counts                                 21
ratio                                         32.26
Name: 90, dtype: object

생각보다 ..판단하기가 어려움.. 그렇다면 ASCII 코드를 MASK 토큰으로 하고 BART를 한다면?

# BART Test

In [152]:
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast, AutoModelForSeq2SeqLM

tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v2')
model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-base-v2')

# tokenizer = PreTrainedTokenizerFast.from_pretrained('kfkas/t5-large-korean-news-title-klue-ynat')
# model = AutoModelForSeq2SeqLM.from_pretrained("kfkas/t5-large-korean-news-title-klue-ynat")

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [196]:
print(tokenizer.all_special_ids)
print(tokenizer.all_special_tokens)

[1, 5, 3, 6]
['</s>', '<unk>', '<pad>', '<mask>']


In [72]:
def replace_ascii_with_mask(text):
    # 결과를 저장할 리스트
    result = []
    
    # 입력 문자열을 한 문자씩 순회
    for char in text:
        # 공백은 그대로 추가
        if char == ' ':
            result.append(char)
        # ASCII 코드 범위(0 ~ 127) 확인, 공백 제외
        elif ord(char) <= 127:
            # ASCII 문자면 <mask>로 대체
            result.append('[MASK]')
        else:
            # 그렇지 않으면 원래 문자를 그대로 추가
            result.append(char)
    
    # 리스트를 문자열로 변환하여 반환
    return ''.join(result)

In [221]:
input_text = "서울문고 N풍그U에 인수… f라인서점 교m영풍y양강d도로"
# input_ids = tokenizer(input_text, return_tensors='pt')['input_ids']
input_ids = tokenizer(replace_ascii_with_mask(input_text), return_tensors='pt')['input_ids']

In [209]:
output_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
output_text

'서울문고  풍그 에 인수...  라인서점 교 영풍 양강 도로'

In [211]:
output_ids = model.generate(
    input_ids, 
    max_length=50, 
    early_stopping=True,
    num_beams=4,
    no_repeat_ngram_size=2,  # 3-그램 이상의 반복 방지
    temperature=0.7,         # 다양성 조절
    top_p=0.92,
    top_k=50
)

# 복원된 텍스트 디코딩
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(output_text)

서울문고  풍그르 에 인수...  라인서점 교보문고, 영풍문고와 양강도로 도로 도로.  도로와 도로의 도로문고가  양천구 신월동 도로에 도로교 


# BERT TEST

Fill-Mask BERT Model Test

In [69]:
from transformers import pipeline, AutoModelForMaskedLM
# monologg/koelectra-base-v3-generator
tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v3-generator", use_fast=True)
model = AutoModelForMaskedLM.from_pretrained("monologg/koelectra-base-v3-generator")
fill_mask = pipeline(
    "fill-mask",
    model=model,
    tokenizer=tokenizer
)
print("MASK Token :",tokenizer.mask_token)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


MASK Token : [MASK]


In [74]:
input_text = "프로야구~롯TKIAs광주 경기 y천취소"
replace_text = replace_ascii_with_mask(input_text)
print(replace_text)
# input_ids = tokenizer(input_text, return_tensors='pt')['input_ids']
predictions = fill_mask(replace_text)

프로야구[MASK]롯[MASK][MASK][MASK][MASK][MASK]광주 경기 [MASK]천취소


In [75]:
def fill_multiple_masks(sentence, fill_mask):
    while "[MASK]" in sentence:
        # 현재 문장에서 첫 번째 [MASK]만 복원
        # print("MASK token Detected, Now Sentence :",sentence)
        predictions = fill_mask(sentence)
        # 가장 높은 확률의 예측을 선택
        best_prediction = predictions[0]
        # 첫 번째 [MASK]를 예측된 단어로 대체
        try:
            sentence = sentence.replace("[MASK]", best_prediction[0]['token_str'], 1)
        except:
            sentence = sentence.replace("[MASK]", best_prediction['token_str'], 1)
        # 복원된 문장 출력
        # print(f"복원 진행 중: {sentence}")
    return sentence

filled_sentence = fill_multiple_masks(replace_text, fill_mask)

MASK token Detected, Now Sentence : 프로야구[MASK]롯[MASK][MASK][MASK][MASK][MASK]광주 경기 [MASK]천취소
복원 진행 중: 프로야구‘롯[MASK][MASK][MASK][MASK][MASK]광주 경기 [MASK]천취소
MASK token Detected, Now Sentence : 프로야구‘롯[MASK][MASK][MASK][MASK][MASK]광주 경기 [MASK]천취소
복원 진행 중: 프로야구‘롯##지[MASK][MASK][MASK][MASK]광주 경기 [MASK]천취소
MASK token Detected, Now Sentence : 프로야구‘롯##지[MASK][MASK][MASK][MASK]광주 경기 [MASK]천취소
복원 진행 중: 프로야구‘롯##지##덕[MASK][MASK][MASK]광주 경기 [MASK]천취소
MASK token Detected, Now Sentence : 프로야구‘롯##지##덕[MASK][MASK][MASK]광주 경기 [MASK]천취소
복원 진행 중: 프로야구‘롯##지##덕##구[MASK][MASK]광주 경기 [MASK]천취소
MASK token Detected, Now Sentence : 프로야구‘롯##지##덕##구[MASK][MASK]광주 경기 [MASK]천취소
복원 진행 중: 프로야구‘롯##지##덕##구)[MASK]광주 경기 [MASK]천취소
MASK token Detected, Now Sentence : 프로야구‘롯##지##덕##구)[MASK]광주 경기 [MASK]천취소
복원 진행 중: 프로야구‘롯##지##덕##구)’광주 경기 [MASK]천취소
MASK token Detected, Now Sentence : 프로야구‘롯##지##덕##구)’광주 경기 [MASK]천취소
복원 진행 중: 프로야구‘롯##지##덕##구)’광주 경기 ##구장천취소


In [76]:
# 입력 텍스트
text = "프로야구‘롯##지##덕##구)’광주 경기 ##구장천취소"

def merge_subwords(text):
    return re.sub(r'\s?##', '', text)

def remove_special_chars(text):
    return re.sub(r'#', '', text)

text = merge_subwords(text)

text = remove_special_chars(text)

# 결과 출력
print("후처리된 텍스트:", text)
# 실제 기사 : 서울문고, 영풍그룹에 인수…오프라인서점 교보-영풍 양강구도로

후처리된 텍스트: 프로야구‘롯지덕구)’광주 경기구장천취소


Noise Data 처리

In [89]:
Noise_text_data = Noise_data[["ID","text","target"]]

In [90]:
Noise_text_data

,ID,text,target
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2
4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6
6,ynat-v1_train_00006,프로야구~롯TKIAs광주 경기 y천취소,1
...,...,...,...
2786,ynat-v1_train_02786,텅 빈 }실 선생님+홀K Y|q% T+X C1,3
2787,ynat-v1_train_02787,13이 노바 라이n2·미J어패0 T3 10 결G상품% *,6
2788,ynat-v1_train_02788,남원소식 춘>X학%단+장Rn 모집,0
2792,ynat-v1_train_02792,"높`X#E율…}BO Q""[/선수 몸값 상승 CAO",1


In [82]:
tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v3-generator", use_fast=True)
model = AutoModelForMaskedLM.from_pretrained("monologg/koelectra-base-v3-generator")
fill_mask = pipeline(
    "fill-mask",
    model=model,
    tokenizer=tokenizer
)
print("MASK Token :",tokenizer.mask_token)

processed_data = []

def fill_multiple_masks(sentence, fill_mask):
    while "[MASK]" in sentence:
        # 현재 문장에서 첫 번째 [MASK]만 복원
        # print("MASK token Detected, Now Sentence :",sentence)
        predictions = fill_mask(sentence)
        # 가장 높은 확률의 예측을 선택
        best_prediction = predictions[0]
        # 첫 번째 [MASK]를 예측된 단어로 대체
        try:
            sentence = sentence.replace("[MASK]", best_prediction[0]['token_str'], 1)
        except:
            sentence = sentence.replace("[MASK]", best_prediction['token_str'], 1)
        # 복원된 문장 출력
    # print(f"복원 데이터: {sentence}")
    return sentence

def merge_subwords(text):
    return re.sub(r'\s?##', '', text)

def remove_special_chars(text):
    return re.sub(r'#', '', text)

def fill_MASK_using_BERT(input_text, fill_mask):
    
    # Fill - Mask
    replace_text = replace_ascii_with_mask(input_text)
    filled_sentence = fill_multiple_masks(replace_text, fill_mask)
    # Post-processing
    post_processed_data = merge_subwords(filled_sentence)
    post_processed_data = remove_special_chars(post_processed_data)
    # 결과 출력
    # print("후처리된 텍스트:", text)
    # print("========================")
    processed_data.append(post_processed_data)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


MASK Token : [MASK]


In [83]:
for noise_text in Noise_text_data["text"]:
    fill_MASK_using_BERT(noise_text, fill_mask)

In [91]:
processed_data

['정씨리파 미사 미사 이용기간 단품 분종보',
 "'찰라국의라국 로한길다 회장  라국송",
 '- 김정숙 자주통일 새정치는이제열은나가야보',
 '오바마,美대선을앞두고 오바마,오바마,단 발오바마에비해 감시 강화',
 '프로야구‘롯지덕구)’광주 경기구장천취소',
 '그의 매력은모·츠·열·나·약 ·나·주가 고··진',
 '아이들고딩수리개 대…맥- 디-디레- 디-디레',
 '문대통령인 당대표의당 민관은동의사위해마에보 철거',
 '추신수 타율 0.1푼지로 시즌 마감…최지만은 2점홈런·홈런홈런·',
 '이는그런수단·팬들고께하는호오는그족 한마 최',
 '현회장차장훈남은 임원을늘려….세. 리. 기보. 육.',
 '○선 폭의 현장조',
 '(2015년레콤 )분기 영,익,,천,,,,…….5％ 증가',
 '생신인증②.안돼제가 유되면 대 가끝',
 '2014년대건덕후강수나라감독 로축구연맹로축구연맹회부',
 "''축구'인천 ''' '''운드 ''''영'",
 '세계적인무역협회 올해는무역인 무역인 천년한번 배출한',
 '박항의 매직은베트남의축구표팀의힘을 쏟고',
 '이는이 는 생산 인공광고에는기보다에는활는적',
 '다저스는버츠 감독은류현전 담당 포수로최 환갑 만들어닝.',
 '박대통령,2월일 연,상,의장 ,견,창조경제 논의',
 '산업생산이십년째 고 제조공장가동률은글로벌금융위주는수합보',
 '그는 유 의는나도못내고사표…특별한번제 흔들',
 "'자농' ,'은행'삼십여명 꺾고딩직직 직",
 '김종민리먼 -라더스 사태처럼 세계경제에 충격이지속죽을 것',
 '이는이자구 김정수트로그보니도모르지는이',
 '인국공은(태왕삼년)의 은(무)은(공)이고왜진보인가',
 '美,여성과한여성 비율은계속 가주가대 입생 중가',
 '위클루돌프스루플코리아루 한국서얼마나 벌을',
 '찍은는 즐거이 가졌다…두개 카메오는즐거는즐거 큐',
 '문대통령 김정태스크트프릴 조속히끝내고 다고합',
 '한·미사이에는 유두바이 치료제 오·솔 美 임· 미수출 본격화',
 '아사이 트-프 외교는험이없다고….오바도는험지-였다',
 '이

In [92]:
Noise_text_data["text"] = processed_data

/tmp/ipykernel_365291/2175804111.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Noise_text_data["text"] = processed_data


In [93]:
Noise_text_data

,ID,text,target
0,ynat-v1_train_00000,정씨리파 미사 미사 이용기간 단품 분종보,4
1,ynat-v1_train_00001,'찰라국의라국 로한길다 회장 라국송,3
2,ynat-v1_train_00002,- 김정숙 자주통일 새정치는이제열은나가야보,2
4,ynat-v1_train_00004,"오바마,美대선을앞두고 오바마,오바마,단 발오바마에비해 감시 강화",6
6,ynat-v1_train_00006,프로야구‘롯지덕구)’광주 경기구장천취소,1
...,...,...,...
2786,ynat-v1_train_02786,텅 빈 채실 선생님은홀로터리,3
2787,ynat-v1_train_02787,이는이 노바 라이어패·미어패 어패· 결상품 상품,6
2788,ynat-v1_train_02788,남원소식 춘제가학단장[UNK] 모집,0
2792,ynat-v1_train_02792,높은박애율…선수 몸값 상승,1


In [94]:
Noise_text_data.to_csv(os.path.join(data_path,"train_Noise_fillmask_data.csv"), encoding="utf-8-sig")